In [66]:
import re
import numpy as np
import pandas as pd
import os, gc
from tqdm.auto import tqdm
from datetime import datetime, timezone, timedelta

import torch
from torch.utils.data import DataLoader,Dataset

In [67]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [68]:
import json
from collections import OrderedDict

In [69]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [70]:
import datasets
import transformers

In [71]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# 토크나이저와 모델 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

model_1 = BartForConditionalGeneration.from_pretrained('D:/VS_code/1st')

model_3 = BartForConditionalGeneration.from_pretrained('D:/VS_code/3st')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [72]:
# DEVICE 초기화
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"DEVICE:{DEVICE}")

# model을 평가모드로 바꿔서 학습이 일어나지 않도록 하고, DEVICE에 전달하기
model.eval()
model = model.to(DEVICE)
model_1 = model_1.to(DEVICE)
model_3 = model_3.to(DEVICE)

DEVICE:cuda


In [73]:
# 원문 데이터 전처리 하는 부분
def token(text):
    text = re.sub('[^!-~ 가-힣]', '', text)
    text = text.replace('  ', '')
    text = tokenizer.encode(text)

    return text

In [74]:
# 전처리된 데이터 요약하는 부분
def summary(text, model, length = 1.0, num = 4, device = DEVICE):
    text = torch.tensor([text]).to(device)
    text = model.generate(text,  length_penalty=length,  num_beams=num,  max_length=512,  eos_token_id=1)
    text = tokenizer.decode(text.squeeze().tolist(), skip_special_tokens=True)

    return text

In [75]:
%cd D:/VS_code/MBC_news

D:\VS_code\MBC_news


In [76]:
####################################################### MBC 스크랩 #######################################################

In [77]:
# MBC에서 긁어올 뉴스 분야
MBC_field = ['정치', '사회', '세계', '경제', '스포츠', 'iMBC 연예']
# MBC 뉴스 링크의 경우, 분야별로 링크에서 장르 부분과, 마지막에 붙는 숫자가 다름
# 따라서 번호뿐만 아니라 어떤 장르의 뉴스를 읽어들일 것인지 함께 함수에서 입력받고,
# 그 분야에 따라 마지막에 붙는 숫자를 변동하기 위해 사전형태로 만들어 둠
MBC_genre = {'politics' : '_36119.html',
             'society' : '_36126.html',
             'world' : '_36133.html',
             'econo' : '_36140.html',
             'sports' : '_36154.html',
             'enter' : '_36161.html'}

In [78]:
# MBC 뉴스 크롤링 후 json 파일 형태로 return 하는 함수
def MBC_data(genre, MBC_number):
  # 입력받은 MBC 뉴스 번호와 장르를 통해 url을 읽어들이고, html 코드를 불러오는 부분
  url = "https://imnews.imbc.com/news/2023/"+genre+"/article/"+str(MBC_number)+MBC_genre[genre]
  result = requests.get(url)
  obj = BeautifulSoup(result.content, "html.parser")

  # 불러온 url에 뉴스가 없을 경우(등록일이 없을 경우 뉴스가 없음) None으로 return 하기
  if (obj.head.find('meta', {'name' : 'nextweb:createDate'}) == None):
    return None
  # 문장 토큰 갯수가 모델 입력 최대 수인 1026개 이상일 경우 건너뛰기
  text = token((((obj.find('div', {'class' : 'news_txt'})).text).replace('\r', ' ')).replace('\n', ''))
  if (len(text) > 1026):
    return None

  # 우리가 필요한 뉴스 분야만 긁어오는 조건문
  if (obj.head.find('meta', {'property' : 'article:section'}).get('content') in MBC_field):
    data = OrderedDict()
    
    # 순서대로 신문사, 제목, 링크, 등록시간, 분야, 작가, 요약문을 dictionary 형태로 저장
    data['image'] = obj.head.find('meta', {'property' : 'og:image'}).get('content')
    data['company'] = 'MBC'
    data['title'] = obj.head.find('meta', {'name' : 'title'}).get('content')
    data['url'] = obj.head.find('meta', {'name' : 'url'}).get('content')
    data['time'] = obj.head.find('meta', {'name' : 'nextweb:createDate'}).get('content')
    data['field'] = obj.head.find('meta', {'property' : 'article:section'}).get('content')
    if (data['field'] == '세계'): data['field'] = '국제'
    # 작가의 경우 기사에 따라 존재하지 않을 때도 있음(대표적으로 날씨 뉴스)
    # 이 때 오류 방지를 위해 None 값이 저장되도록 설정 (MBC의 경우 날씨 기자가 iMBC 연예 같은 신문사 계정일 때도 있음)
    if (obj.head.find('meta', {'name' : 'author'}) == None):
      data['writer'] = None
    else:  
      data['writer'] = obj.head.find('meta', {'name' : 'author'}).get('content')
    # 긁어온 데이터에서 원문에 해당하는 부분을 바로 summary 함수에 넣어서 요약문을 저장하도록 설정
    data['text_short'] = summary(text, model_1, length=2.0, num=8)
    data['text_middle'] = summary(text, model, length=2.0, num=8)
    data['text_long'] = summary(text, model_3)
    
    return data

In [79]:
####################################################### 실시간 스크랩 #######################################################

In [80]:
# MBC 실시간 웹크롤링 코드
# MBC의 경우 분야별로 링크가 다른만큼 분야별로 최근 기사의 번호가 다름
# 이를 위해 최근 기사의 번호를 분야별로 각각 저장할 필요가 있어서 dictionary 형태로 저장
MBC_number = {'politics' : 6485026,
             'society' : 6484930,
             'world' : 6484921,
             'econo' : 6485522,
             'sports' : 6485710,
             'enter' : 6484884}

while True:
  # MBC_genre의 key 값별로 data 스크랩 하기 위한 for문 선언
  for genre in MBC_genre:
      data = MBC_data(genre, MBC_number[genre])

      # data 체크
      if (data == None):
        # 스크랩한 번호부터 1000까지 순차적으로 더하면서 사이 번호를 건너뛰고 생성된 뉴스가 있는지 체크
        for count in range(1000):
          data = MBC_data(genre, MBC_number[genre] + count)
          # 만약 생성된 뉴스가 있다면 MBC_number 값을 해당 뉴스 번호로 저장하고, 반복문 탈출 / 변동된 값은 사전을 업데이트 시키며 완료됨
          if (data != None):
            MBC_number[genre] = MBC_number[genre] + count
            break
        # 시스템 과열 방지를 위해 30초간 휴식시키는 부분, 작동 체크를 위한 선 출력 (MBC는 최대 600개의 링크를 체크해야하여 30초만 휴식시키기로 함)
        print('Sleep) Genre, MBC_N : ', genre,', ', MBC_number[genre])
        #time.sleep(30)
      else:
        # data가 null 값이 아니라면, json 파일로 뽑아내서 저장함
        # 파일명은 '(genre)_(MBC_number)_MBC_news.json' 형태로 저장 (genre는 해당 분야 영어명, MBC_number는 숫자)
        with open(str(MBC_number[genre])+'_'+genre+'_MBC_news.json', 'w', encoding='utf-8') as make_file:
          json.dump(data, make_file, ensure_ascii=False, indent='\t')
        print('---------------------------------')
        print('Genre, MBC_N : ', genre,', ', MBC_number[genre])
        # 이미 한번 체크한 번호니까 다음 번호부터 체크하도록 +1 하기 / 다음 분야 링크 체크하는 걸로 넘어가게 됨
        MBC_number[genre] += 1

---------------------------------
Genre, MBC_N :  politics ,  6484707
---------------------------------
Genre, MBC_N :  society ,  6484735
---------------------------------
Genre, MBC_N :  world ,  6484646
---------------------------------
Genre, MBC_N :  econo ,  6484676
---------------------------------
Genre, MBC_N :  sports ,  6484622
---------------------------------
Genre, MBC_N :  enter ,  6484690
Sleep) Genre, MBC_N :  politics ,  6484716
Sleep) Genre, MBC_N :  society ,  6484754
Sleep) Genre, MBC_N :  world ,  6484654
Sleep) Genre, MBC_N :  econo ,  6484703
Sleep) Genre, MBC_N :  sports ,  6484698
Sleep) Genre, MBC_N :  enter ,  6484722
---------------------------------
Genre, MBC_N :  politics ,  6484716
---------------------------------
Genre, MBC_N :  society ,  6484754
---------------------------------
Genre, MBC_N :  world ,  6484654
---------------------------------
Genre, MBC_N :  econo ,  6484703
---------------------------------
Genre, MBC_N :  sports ,  6484698
-----

KeyboardInterrupt: 